# ML Trader

In [1]:
from utils.packages import *
from utils.variables import *
from utils.i_o import *
from utils.dir_slope import *
from utils.engine import *
from utils.loops import *
from utils.order import *
import winsound

In [ ]:
%%time
data = run_engine(data)

In [ ]:
generate_result_report(data)

data['df']['long_profit'] = np.nan
data['df']['long_loss'] = np.nan
data['df']['short_profit'] = np.nan
data['df']['short_loss'] = np.nan

data['df']['long_profit'][data['df']['pl'] > 0] = data['df']['long_close']
data['df']['long_loss'][data['df']['pl'] < 0] = data['df']['long_close']

data['df']['short_profit'][data['df']['pl'] > 0] = data['df']['short_close']
data['df']['short_loss'][data['df']['pl'] < 0] = data['df']['short_close']

del data['long_close']
del data['short_close']

In [ ]:
data["plot"] = True
data['plot_type'] = 'file'
data['reduce_plot'] = True
data['plot_start']  = '2021-01-7'
data['plot_stop']   = '2021-01-8'
# data['things_to_plot'] = ['tick', 'lema', 'h_l_lema', 'tick_angle', 'positions']
data['things_to_plot'] = ['tick', 'lema', 'h_l_lema', 'positions']
data['things_to_plot'] = ['tick', 'lema', 'h_l_lema', 'tick_angle', 'positions']

In [ ]:
plot_graph(data)

data['df'] = pd.read_csv('data/ema_df-(2021-2021)-(1-1)-(1-31).csv')

data['angle_len'] = 10
data = get_x_axis(data)
data['df']['tick_angle'] = data['df']['lema'].rolling(window=data['angle_len']).progress_apply(roll_slope)
data['df'] = data['df'].dropna()

data['df'] = data['df'][data['columns_list']]

data['df'].to_csv('data/ema_df-(2021-2021)-(1-1)-(1-31).csv')
data['df'].head(2)

In [ ]:
data['report_df'][data['report_df']['date_val'] == 7][0:40]

# -------------------------------

In [138]:
x = 25000
data['df'] = pd.read_csv('data/eurusd_2021.csv', nrows = 10000, skiprows = 75000)

cols = ['DateTime', 'Bid', 'Ask', 'Volume']
data['df'] = data['df'].set_axis(cols, axis=1, inplace=False)

data['df']['DateTime_frmt']   = [dt.datetime.strptime(x.split(".")[0],"%Y%m%d %H:%M:%S") for x in data['df']['DateTime']]
data['df']['tick'] = data['df'][['Ask', 'Bid']].mean(axis=1)
data['df']['spread'] = data['df']['Ask'] - data['df']['Bid']
data['df']['ask_diff'] = data['df']['Ask'].diff()
data['df']['bid_diff'] = data['df']['Bid'].diff()
data['df'] = data['df'].dropna()

# data['df']['diff_peak'] = np.nan
# diff_limit = 0.00004
# data['df']['diff_peak'][(abs(data['df']['bid_diff']) >= diff_limit) 
#                         & (abs(data['df']['ask_diff']) >= diff_limit)] = data['df']['tick']

data['df']['avg_volume'] = data['df']['Volume'].mean()

data['df']['diff_peak'] = np.nan
diff_limit = 0.00004
data['df']['diff_peak'][(abs(data['df']['bid_diff']) >= diff_limit) 
                        & (abs(data['df']['ask_diff']) >= diff_limit)
                       & (data['df']['Volume'] < data['df']['avg_volume'])] = data['df']['tick']


data['df'] = data['df'][['DateTime_frmt', 'Bid', 'bid_diff', 'Ask', 'ask_diff','tick', 'spread','Volume', 'avg_volume','diff_peak']]

data['df'].head(2)

,DateTime_frmt,Bid,bid_diff,Ask,ask_diff,tick,spread,Volume,avg_volume,diff_peak
1,2021-01-04 15:02:31,1.22875,0.00000,1.22879,0.00001,1.22877,0.00004,1,2.618662,NaN
2,2021-01-04 15:02:32,1.22876,0.00001,1.22880,0.00001,1.22878,0.00004,1,2.618662,NaN


In [140]:
 # Plot Layout --------------------------------
chart_name = f"Trade Chart"
layout = go.Layout(title = chart_name,
                xaxis = dict(title="DateTime"),
                xaxis2 = dict(title= 'x', side= 'top'),

                yaxis = dict(title="PIP"),
                yaxis2 = dict(title= 'Trend Angle', overlaying="y", side="right",)
                )
fig = go.Figure(layout = layout)
# -------------------------------------------------------------------


# -------------------------------------------------------------------

fig.add_trace(go.Scatter(x=data['df']['DateTime_frmt'],
                        y=data['df']['Ask'],
                        mode='lines',
                        name='ask',
                        line=dict(color='red', width=1),
                    )
            )

fig.add_trace(go.Scatter(x=data['df']['DateTime_frmt'],
                        y=data['df']['Bid'],
                        mode='lines',
                        name='bid',
                        line=dict(color='blue', width=1),
                    )
            )

fig.add_scatter(x = data['df']['DateTime_frmt'], 
            y = data['df']['diff_peak'], 
            mode = 'markers', 
            name = 'diff_peak',
            marker_symbol = 'triangle-up',
            marker=dict(color='cadetblue',
                        size=10,
                        line=dict(
                            color='black',
                            width=1
                        )),
            opacity=1)

# fig.add_trace(go.Scatter(x=data['df']['DateTime_frmt'],
#                                 y=data['df']['Volume'],
#                                 mode='lines',
#                                 name='Volume',
#                                 yaxis='y2',
#                             line=dict(color='lightblue', width=1),
#                             )
#                     )     

# fig.add_trace(go.Scatter(x=data['df']['DateTime_frmt'],
#                                 y=data['df']['avg_volume'],
#                                 mode='lines',
#                                 name='avg_volume',
#                                 yaxis='y2',
#                             line=dict(color='blue', width=1),
#                             )
#                     )  

# fig.add_trace(go.Scatter(x=data['df']['DateTime_frmt'],
#                                 y=data['df']['ask_diff'],
#                                 mode='lines',
#                                 name='ask_diff',
#                                 yaxis='y2',
#                             line=dict(color='Red', width=1),
#                             )
#                     )    
    
chart_name = str(dt.datetime.now())
chart_name = chart_name.replace(":", "-")
chart_name = chart_name.replace(".", "-")
chart_name = chart_name.replace(" ", "-")
data['chart_file_path'] = (f'{os.getcwd()}\\data\\chart-{chart_name}.html')

fig.write_html(data['chart_file_path'])
webbrowser.get(data['chrome_path']).open(data['chart_file_path'])

True

In [113]:
data['df'][data['df']['DateTime_frmt'] > '2021-01-03 23:43:10'][0:40]

,DateTime_frmt,Bid,bid_diff,Ask,ask_diff,tick,spread,Volume,diff_peak
1,2021-01-04 15:02:31,1.22875,0.00000,1.22879,0.00001,1.228770,0.00004,1,NaN
2,2021-01-04 15:02:32,1.22876,0.00001,1.22880,0.00001,1.228780,0.00004,1,NaN
3,2021-01-04 15:02:32,1.22878,0.00002,1.22880,0.00000,1.228790,0.00002,1,NaN
4,2021-01-04 15:02:32,1.22877,-0.00001,1.22879,-0.00001,1.228780,0.00002,2,NaN
5,2021-01-04 15:02:33,1.22877,0.00000,1.22878,-0.00001,1.228775,0.00001,1,NaN
6,2021-01-04 15:02:33,1.22875,-0.00002,1.22878,0.00000,1.228765,0.00003,1,NaN
7,2021-01-04 15:02:33,1.22874,-0.00001,1.22877,-0.00001,1.228755,0.00003,1,NaN
8,2021-01-04 15:02:33,1.22873,-0.00001,1.22876,-0.00001,1.228745,0.00003,1,NaN
9,2021-01-04 15:02:34,1.22875,0.00002,1.22877,0.00001,1.228760,0.00002,1,NaN
10,2021-01-04 15:02:34,1.22875,0.00000,1.22878,0.00001,1.228765,0.00003,1,NaN
